Created a connection with google Drive so it can access the downloaded Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Step 1| Access Data:
Accessing the dataSet downloaded from Kaggle (contains 16 mil chess positions)


In [ ]:

from pandas import read_csv
csv = read_csv('gdrive/MyDrive/chessData.csv') #change here for dataset



Step 2 | Prepare data: using the python chess library to convert a chess board in a matrix of strings (to be fed into a neural network)


In [ ]:
chess_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0],
}
def make_matrix(board): 
    pgn = board.epd()
    foo = []  
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []  
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append('.')
            else:
                foo2.append(thing)
        foo.append(foo2)
    return foo

def translate(matrix,chess_dict):
    rows = []
    for row in matrix:
        terms = []
        for term in row:
            terms.append(chess_dict[term])
        rows.append(terms)
    return rows

Gets values and performs two sets of operations so it can be fed to neural network

In [ ]:
import chess
import numpy as np
fen = csv['FEN'].values
values = csv['Evaluation'].values
length = 10000
X =[]
y= values[:length]
defects = []
for i in range(length):
    board = chess.Board(fen[i])
    matrix = make_matrix(board.copy())
    translated = translate(matrix,chess_dict)
    X.append(translated)

For every occurence of a hash, multiple the last value in the string by 1000.

In [ ]:
for i in range(length):
    if '#' in y[i]:
        y[i] = float(y[i][-1]) * 1000
y = y.astype('float32')

Normalizing the data

In [ ]:
minimum = min(y)
maximum = max(y)
for i in range(len(y)):
    y[i] = (y[i]-minimum)/(maximum-minimum)

Step 3| Create and train the neural network:

In [ ]:
from keras import callbacks, optimizers
from keras.layers import (LSTM, BatchNormalization, Dense, Dropout, Flatten,TimeDistributed)
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model, model_from_json
from IPython.display import clear_output
from matplotlib import pyplot as plt
model = Sequential()
model.add(Conv2D(filters=10, kernel_size=1, activation='relu', input_shape=(8,8,12)))
model.add(MaxPooling2D(pool_size=2, strides=None))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(1,activation = 'sigmoid'))

model.compile(optimizer = 'Adam',loss='mse')
h5 = 'chess' + '_best_model' + '.h5'
checkpoint = callbacks.ModelCheckpoint(h5,
                                           monitor='val_loss',
                                           verbose=0,
                                           save_best_only=True,
                                           save_weights_only=True,
                                           mode='auto',
                                           period=1)
es = callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5000/10)
callback = [checkpoint,es]
json = 'chess' + '_best_model' + '.json'
model_json = model.to_json()
with open('gdrive/MyDrive/chess_best_model.json', "w") as json_file: #change here for model
    json_file.write(model_json)
print('Training Network...')

test = 1000
X = np.array(X)
X_train = X[test:]
y_train = y[test:]
X_test = X[:test]
y_test = y[:test]
model.fit(X_train,y_train,epochs = 1000,verbose = 2,validation_data = (X_test,y_test),callbacks = callback)

Training Network...
Epoch 1/1000
282/282 - 2s - loss: 0.0210 - val_loss: 0.0051 - 2s/epoch - 7ms/step
Epoch 2/1000
282/282 - 1s - loss: 0.0040 - val_loss: 0.0035 - 1s/epoch - 4ms/step
Epoch 3/1000
282/282 - 1s - loss: 0.0029 - val_loss: 0.0025 - 1s/epoch - 4ms/step
Epoch 4/1000
282/282 - 1s - loss: 0.0027 - val_loss: 0.0024 - 1s/epoch - 4ms/step
Epoch 5/1000
282/282 - 1s - loss: 0.0026 - val_loss: 0.0023 - 975ms/epoch - 3ms/step
Epoch 6/1000
282/282 - 1s - loss: 0.0025 - val_loss: 0.0024 - 976ms/epoch - 3ms/step
Epoch 7/1000
282/282 - 1s - loss: 0.0024 - val_loss: 0.0023 - 1s/epoch - 4ms/step
Epoch 8/1000
282/282 - 1s - loss: 0.0024 - val_loss: 0.0023 - 1s/epoch - 4ms/step
Epoch 9/1000
282/282 - 1s - loss: 0.0024 - val_loss: 0.0023 - 970ms/epoch - 3ms/step
Epoch 10/1000
282/282 - 1s - loss: 0.0024 - val_loss: 0.0023 - 1s/epoch - 4ms/step
Epoch 11/1000
282/282 - 1s - loss: 0.0024 - val_loss: 0.0021 - 981ms/epoch - 3ms/step
Epoch 12/1000
282/282 - 1s - loss: 0.0024 - val_loss: 0.0025 - 9

Step 4| Load Model: based on the best weights


In [ ]:
from keras.models import Sequential, load_model, model_from_json
def load_keras_model(dataset,loss,optimizer):
        json_file = open('gdrive/MyDrive/chess_best_model.json', 'r',encoding="utf-8-sig") #change here for model
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        model.compile(optimizer=optimizer, loss=loss, metrics = None)
        model.load_weights(dataset+'_best_model'+'.h5')
        return model

Step 5| Monte Carlo Algorithm:
-algorithm will choose random legal moves
-a numerical evaluation will be calculated that will be added to the score of the move
-the highest scored move will be played

In [ ]:
import random
import chess
board = chess.Board()
moves = []
model = load_keras_model('chess','mse','Adam')

def calculate_move(depth, board, epochs):
    first_legal_moves = str(board.legal_moves)[38:-2].replace(',','').split()
    if len(first_legal_moves[0])==1:
      first_legal_moves[0]=first_legal_moves[0]
    else:
      first_legal_moves[0]=first_legal_moves[0][1:]
    scores = [[0]] * len(first_legal_moves)
    for epoch in range(epochs):
        for first_move in range(len(first_legal_moves)):
            play_board = board.copy()
            play_board.push_san(first_legal_moves[first_move])
            for _ in range(depth):
                legal_moves = str(play_board.legal_moves)[38:-2].replace(',','').split()
                if len(legal_moves[0])==1:
                  legal_moves[0]=legal_moves[0]
                else:
                  legal_moves[0]=legal_moves[0][1:]
                if len(legal_moves)>1:
                    move = random.choice(legal_moves)
                    play_board.push_san(move)
                else:
                    break
            translated = np.array(translate(matrix, chess_dict))
            scores[first_move] += model.predict(translated.reshape(1, 8, 8, 12)) * (maximum - minimum) + minimum
    print(first_legal_moves[scores.index(max(scores))])
    return first_legal_moves[scores.index(max(scores))]

Play against the AI:

In [ ]:
while True:
    legal_moves = str(board.legal_moves)[38:-2].replace(',','').split()
    print(board)
    print(str(board.legal_moves)[38:-2].replace(',','').split())
    move = input('Which move do you want to Play?')
    board.push_san(move)
    print(board)
    matrix = make_matrix(board.copy())
    translated = np.array(translate(matrix,chess_dict))
    print(model.predict(translated.reshape(1,8,8,12)))

    move = calculate_move(10, board, 10)
    board.push_san(move)
 

    

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
['(Nh3', 'Nf3', 'Nc3', 'Na3', 'h3', 'g3', 'f3', 'e3', 'd3', 'c3', 'b3', 'a3', 'h4', 'g4', 'f4', 'e4', 'd4', 'c4', 'b4', 'a4']
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . P . .
P P P P P . P P
R N B Q K B N R
[[0.42247552]]
Nh6
r n b q k b . r
p p p p p p p p
. . . . . . . n
. . . . . . . .
. . . . . . . .
. . . . . P . .
P P P P P . P P
R N B Q K B N R
['(Nh3', 'Kf2', 'Nc3', 'Na3', 'f4', 'h3', 'g3', 'e3', 'd3', 'c3', 'b3', 'a3', 'h4', 'g4', 'e4', 'd4', 'c4', 'b4', 'a4']
r n b q k b . r
p p p p p p p p
. . . . . . . n
. . . . . . . .
. . . . P . . .
. . . . . P . .
P P P P . . P P
R N B Q K B N R
[[0.42640945]]
Rg8
r n b q k b r .
p p p p p p p p
. . . . . . . n
. . . . . . . .
. . . . P . . .
. . . . . P . .
P P P P . . P P
R N B Q K B N R
['(Nh3', 'Ne2', 'Ba6', 'Bb5', 'Bc4', 'Bd3', 'Be2', 'Kf2', 'Ke2', 'Qe2', 'Nc3

Reference :
@website{CODE,

  author={Victor S.},

  title={Creating a Chess Algorithm using Deep Learning and Monte Carlo Methods}, 

  year={2020},

  volume={},

  number={},
  
  url={https://towardsdatascience.com/creating-a-chess-algorithm-using-deep-learning-and-monte-carlo-methods-d7dabd275e63}
}